## ViVa VCF Visualizer - Version 0.2


## Upload VCF and choose visualization settings

In [9]:
vcf_filename = "variants.filtered.191_joint.vcf"
field_to_visualize = "genotype" # field(s) = "read_depth", *etc.
variant_filter = "pass_only" , "list", "significantList_for_proteinstructures.csv"#"range", "chr1:20000000-30000000"
sample_filter = "reorder_columns", "sample_phenotype_matrix.csv", "case_control_status" # "select_columns", "select_column_list.txt",
save_format = "pdf"
plot_title = "title_here"
;

#issue - if select only one filter - error occurs because cant take size(String,1)
#use for variant_filter[1:end]




## Once settings are chosen, run notebook.

In [11]:
using DataFrames #use CSV.jl ? depwarnings
using CSV
using PlotlyJS
using Rsvg
using Blink
using ViVa

@time ViVa.jupyter_main(vcf_filename,field_to_visualize,variant_filter,sample_filter,save_format,plot_title)

ParseError("missing last argument in \"1:\" range expression ")
Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] #readtable#229(::Bool, ::Char, ::Array{Char,1}, ::Char, ::Array{String,1}, ::Array{String,1}, ::Array{String,1}, ::Bool, ::Int64, ::Array{Symbol,1}, ::Array{Any,1}, ::Bool, ::Char, ::Bool, ::Int64, ::Array{Int64,1}, ::Bool, ::Symbol, ::Bool, ::Bool, ::DataFrames.#readtable, ::String) at /Users/George/.julia/v0.6/DataFrames/src/deprecated.jl:1045
 [3] (::DataFrames.#kw##readtable)(::Array{Any,1}, ::DataFrames.#readtable, ::String) at ./<missing>:0
 [4] load_vcf(::String) at /Users/George/projects_code/ViVa.jl/src/vcf_utils.jl:62
 [5] jupyter_main(::String, ::String, ::String, ::Tuple{String,String,String}, ::String, ::String) at /Users/George/projects_code/ViVa.jl/src/notebook_utils.jl:11
 [6] include_string(::String, ::String) at ./loading.jl:515
 [7] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /Users/George/.julia/v0.6/IJulia/src/execute_request.jl

LoadError: [91mMethodError: no method matching size(::String, ::Int64)[0m
Closest candidates are:
  size(::Any, ::Integer, [91m::Integer[39m, [91m::Integer...[39m) where N at abstractarray.jl:30
  size([91m::Char[39m, ::Any) at char.jl:14
  size([91m::BitArray{1}[39m, ::Any) at bitarray.jl:66
  ...[39m

# These are the global variables you must define before running the notebook:
### Required options:  

#### `vcf_filename :: String` - This is a vector of filters that are applied to rows from the VCF file for visualization

#### `field_to_visualize :: String` - This is a vector of filters that are applied to rows from the VCF file for visualization

#### `variant_filter :: Vector{String}` - This is a vector of filters that are applied to rows from the VCF file for visualization
#### e.g. 
`variant_filter = ["pass_only", "chr1:20000000-30000000"]`

Variant_filter input options:
- `"all"` = no filter, visualize all variants
- `"pass_only"` = filter to select variant rows that pass samtools QC
- `"list.txt"` = select variants that match a user produced list of variant chromosome positions
- `"chr1:20000000-30000000"` = select variants within a genomic range in format "chr1:20000000-30000000"

Sample_filter input options:
- `"select_columns"` also requires -> `"sample_id_list.txt"`
- `"reorder_columns"` also requires -> `"phenotype_matrix.csv"` & `"phenotype_label_to_sort_by"`

#### `sample_filter :: Vector{String}` - This is a vector of filters that are applied to rows from the VCF file for visualization

